<a href="https://colab.research.google.com/github/Taruneswar37/Movie-Review-Sentiment-Analysis/blob/main/IMDB_Movie_Review_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install kaggle

Importing the dependecies


In [3]:
import os
import json
from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Data collection through kaggle API

In [5]:
kaggle_dict=json.load(open("kaggle.json"))

In [6]:
kaggle_dict.keys()

dict_keys(['username', 'key'])

In [7]:
#setup kaggle credential as environent variable
os.environ["KAGGLE_USERNAME"]=kaggle_dict["username"]
os.environ["KAGGLE_KEY"]=kaggle_dict["key"]

In [8]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews --force

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 58% 15.0M/25.7M [00:00<00:00, 154MB/s]
100% 25.7M/25.7M [00:00<00:00, 187MB/s]


In [9]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [10]:
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
  zip_ref.extractall()

In [11]:
!ls


'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


Load the Dataset


In [12]:
data=pd.read_csv("/content/IMDB Dataset.csv")

In [13]:
data.shape

(50000, 2)

In [14]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [15]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [16]:
data["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


Converting the categorical valueinto numerical value

In [17]:
data.replace({"sentiment":{"positive":1,"negative":0}},inplace=True)

<ipython-input-17-329ec005e21e>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment":{"positive":1,"negative":0}},inplace=True)


In [19]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [20]:
data["sentiment"].value_counts()

,count
sentiment,
1,25000
0,25000


In [21]:
#split the data for train and test
train_data,test_data=train_test_split(data,test_size=0.2,random_state=42)

In [22]:
print(train_data.shape)
print(test_data.shape)


(40000, 2)
(10000, 2)


Data preprocessing

In [23]:
#Tokenize the data
tokenizer=Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train=pad_sequences(tokenizer.texts_to_sequences(train_data["review"]),maxlen=200)
X_test=pad_sequences(tokenizer.texts_to_sequences(test_data["review"]),maxlen=200)

In [24]:
print(X_train)
print(X_test)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]
[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [25]:
Y_train=train_data["sentiment"]
Y_test=test_data["sentiment"]

In [26]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


In [27]:
print(Y_test)

33553    1
9427     1
199      0
12447    1
39489    0
        ..
28567    0
25079    1
18707    1
15200    0
5857     1
Name: sentiment, Length: 10000, dtype: int64


LSTM Model

In [28]:
model=Sequential()
model.add(Embedding(input_dim=5000,output_dim=128,input_shape=(200,)))
model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [29]:
model.build()
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 200, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [31]:
model.fit(X_train,Y_train,epochs=5,batch_size=64,validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 317s 627ms/step - accuracy: 0.7307 - loss: 0.5232 - val_accuracy: 0.8519 - val_loss: 0.3606
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 320s 623ms/step - accuracy: 0.8618 - loss: 0.3388 - val_accuracy: 0.8340 - val_loss: 0.3760
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 325s 628ms/step - accuracy: 0.8776 - loss: 0.3006 - val_accuracy: 0.8699 - val_loss: 0.3087
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 315s 614ms/step - accuracy: 0.8900 - loss: 0.2740 - val_accuracy: 0.8590 - val_loss: 0.3373
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 327s 625ms/step - accuracy: 0.9113 - loss: 0.2297 - val_accuracy: 0.8721 - val_loss: 0.3119


Model Evaluation

In [32]:
loss,accuracy=model.evaluate(X_test,Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 28s 87ms/step - accuracy: 0.8792 - loss: 0.2950
Test Loss: 0.2973887622356415
Test Accuracy: 0.8781999945640564


Building a Predictive system

In [44]:
def predict_sentiment(review):
  sequence=tokenizer.texts_to_sequences([review])
  padded_sequence=pad_sequences(sequence,maxlen=200)
  prediction=model.predict(padded_sequence)
  sentiment="positive" if prediction[0][0]>0.5 else "negative"
  return sentiment

In [65]:
new_review="This is the fantastic movie"
sentiment=predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
The sentiment of the review is: positive


In [61]:
new_review="This movie is awesome"
sentiment=predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
The sentiment of the review is: positive


In [36]:
new_review="This movie is good but not expected"
sentiment=predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
The sentiment of the review is: negative


In [66]:
new_review="This is the worst movie i have ever seen"
sentiment=predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
The sentiment of the review is: negative
